
For this example, we using PyPI package: https://pypi.org/project/ollama/

Here you can find official documentation for `Ollama` Python library: https://github.com/ollama/ollama-python

Here is official documentation how to install `Ollama`: https://ollama.com/download

---
Let's start from simple things. What models are available to us?

It's very easy to do using `Ollama`.

In [ ]:
import ollama

# This will return a list of all downloaded previous models.
print(ollama.list())

Let's make it adequately formatted and easy to read.

In [ ]:

for model in ollama.list()["models"]:
    print(f"- '{model['name']}' of family '{model['details']['family']}', in format '{model['details']['format']}'")

---
Let's pick some simple model for our examples. I think that small model of `qwen2` family should work fine for simple text generatin examples.

- [qwen2](https://ollama.com/library/qwen2): `qwen2:0.5b`, `qwen2:1.5b`

In [ ]:
USE_MODEL = "qwen2:0.5b"

If this model is on the list above, then there are no problems. We should be able to use it.

Let's try to generate answer for a simple question.

In [ ]:
response = ollama.generate(USE_MODEL, 'Why is the sky blue?')
print(response['response'])

---
But sadly, we don't have that model downloaded, and we need to download it first to start work with that model.

To do that, the easiest way is to use `ollama.pull`.

In [ ]:
ollama.pull(USE_MODEL)

---
Note that here we using small model for simplicity and generation speed.

To get more reasonable and consistent answers, consider using `mistral:7b` or `llama3:8b`.

And now we should be able to get our answer.

In [ ]:
response = ollama.generate(USE_MODEL, 'Why is the sky blue?')
print(response['response'])

---
Exactly the same example as above but with streaming ability; in this case, LLM will generate an answer as a typewriter.

In [ ]:
stream = ollama.generate(USE_MODEL, 'Why is the sky blue?', stream=True)
for chunk in stream:
  # by default `end` is set to `'\n'` and `flush` is not set.
  # print("...", end='\n')
  print(chunk["response"], end='', flush=True)

---
Note that `ollama.generate` is, in reality, `client.generate` and `client` is an instance of `ollama.Client` with no parameters.

That's how creating all these functions looks in the `ollama` module.

```python

_client = Client()

generate = _client.generate
chat = _client.chat
embeddings = _client.embeddings
pull = _client.pull
push = _client.push
create = _client.create
delete = _client.delete
list = _client.list
copy = _client.copy
show = _client.show
ps = _client.ps
```

Then, by using `ollama.list`, `ollama.pull` or `ollama.generate` or any other method, you use the methods of the `Client` class instance.

However, if the `ollama` or `ollama-docker` container is not installed on the current machine, there can be a problem.

It's impossible to set a `host` parameter to define a host with which to interact.

---
To solve the problem with the configuration of the host, we can use the `Client` class directly to create a client instance and set the `host` parameter.

If the `Ollama` server is installed on the current machine, just use `http://localhost:11434`; for the remote machine, you will need to point to the exact machine address/IP and port. example `http://192.168.1.100:11434`.

In [ ]:
client = ollama.Client(host='http://localhost:11434') # In this example, we have `Ollama` installed locally.

# Here we using `ollama.generate` but with our own instance of `ollama.Client`
response = client.generate(USE_MODEL, 'Why is the sky blue?')

print(response['response'])

The same example again uses streaming instead of waiting until the whole answer is generated.

Note that the structure of `chunk` has changed, and we now need to use these keys to access the current message part `chunk['message']['content']`.

In [ ]:
client = ollama.Client(host='http://localhost:11434')

stream = client.chat(
    model='qwen2:0.5b',
    messages=[{'role': 'user', 'content': 'Why is the sky blue?'}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)
